In [150]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# remove scientific notation display for floats
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Data Sources and Notes:

## Box Office Mojo

### Box Office Mojo movie grosses

In [199]:
#read data
gross_bom = pd.read_csv('zippedData/bom.movie_gross.csv.gz', compression='gzip')
gross_bom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
title             3387 non-null object
studio            3382 non-null object
domestic_gross    3359 non-null float64
foreign_gross     2037 non-null object
year              3387 non-null int64
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


## Note: Not all movies have information for foreign gross earnings. To suplement foreign gross when not available we will use the ratio of the average (median) domestic to foreign gross, and then make a column for total gross earnings

In [200]:
# Foreign Gross is a string accounting for NaN.
# Creating foreign_release column in order to create expected foreign gross column.
gross_bom['no_foreign_release'] = gross_bom.foreign_gross.map(lambda x : pd.isnull(x))

In [201]:
print(gross_bom['no_foreign_release'].value_counts())
gross_bom.foreign_gross.isna().sum()

False    2037
True     1350
Name: no_foreign_release, dtype: int64


1350

In [202]:
# remove na rows from foreign_gross in order to calculate stats
no_na_foreign_gross = gross_bom.dropna(axis= 0, subset=['foreign_gross'])

In [204]:
# remove comma from string then convert to int
no_na_foreign_gross.foreign_gross = no_na_foreign_gross.foreign_gross.map(lambda val : int(float(val.replace(",", ""))))

In [205]:
no_na_foreign_gross.agg(['mean', 'median', 'std'])

,domestic_gross,foreign_gross,year,no_foreign_release
mean,46973114.37780,74872810.14973,2013.49288,0.00000
median,16500000.00000,18700000.00000,2013.00000,0.00000
std,81599661.10596,137410600.84191,2.59241,0.00000


In [206]:
# find ratio of domestic to foreign mean
no_na_foreign_gross.domestic_gross.mean()/no_na_foreign_gross.foreign_gross.mean()

0.6273721299342642

In [207]:
# find ratio of domestic to foreign median
no_na_foreign_gross.domestic_gross.median()/no_na_foreign_gross.foreign_gross.median()

0.8823529411764706

In [209]:
# set foreign_gross na's to 0 in order to do arithmetic operations
gross_bom.foreign_gross.fillna("0", inplace=True)

In [218]:
# convert foreign_gross strings to int
gross_bom.foreign_gross = gross_bom.foreign_gross.map(lambda val : int(float(val.replace(",", ""))))

False

In [232]:
no_na_data = {'dom_median': no_na_foreign_gross.domestic_gross.median(), 'foreign_median': no_na_foreign_gross.foreign_gross.median()}

In [236]:
def total_gross(no_foreign_release, domestic_gross, foreign_gross):
    if no_foreign_release:
        return domestic_gross*(no_na_data['foreign_median']/no_na_data['dom_median'])
    else:
        return domestic_gross + foreign_gross

In [237]:
gross_bom['total_gross'] = gross_bom.apply(lambda row : total_gross(row.no_foreign_release, row.domestic_gross, row.foreign_gross), axis=1)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

# <span style="color:red"> TODO: Make total gross column </span>

## IMDB
Note: foreign keys look like nconst is for Person Name, and tconst is for Movie Title

### IMDB Basic info by Name

In [56]:
name_basics_imdb = pd.read_csv('zippedData/imdb.name.basics.csv.gz', compression='gzip')
name_basics_imdb.head(1)

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"


### IMDB some kind of info by Title
# <span style="color:red"> I think this is related to international releases? I can't tell yet </span>

In [57]:
title_imdb = pd.read_csv('zippedData/imdb.title.akas.csv.gz', compression='gzip')
title_imdb.head(2)

,title_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,NaN,NaN,0.0
1,tt0369610,11,Jurashikku warudo,JP,NaN,imdbDisplay,NaN,0.0


### IMDB Basic info by Title

In [126]:
title_basics_imdb = pd.read_csv('zippedData/imdb.title.basics.csv.gz', compression='gzip')
title_basics_imdb.head(1)

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"


# <span style="color:red"> multiple genres are contained in a string </span>

### IMDB Crew (Writers and Directors)
This looks to be a connecting table

In [127]:
crew_imdb = pd.read_csv('zippedData/imdb.title.crew.csv.gz', compression='gzip')
crew_imdb.head(1)

,tconst,directors,writers
0,tt0285252,nm0899854,nm0899854


### IMDB Principles
(Actors, directors, producers, editors, actress, composer, cinematographer, writer, self, production designer, archive footage, archive sound )

In [66]:
title_principals_imdb = pd.read_csv('zippedData/imdb.title.principals.csv.gz', compression='gzip')
title_principals_imdb.head(1)

,tconst,ordering,nconst,category,job,characters
0,tt0111414,1,nm0246005,actor,NaN,"[""The Man""]"


### IMDB Rating
Key for titles along with rating and number of votes

In [69]:
title_ratings_imdb = pd.read_csv('zippedData/imdb.title.ratings.csv.gz', compression='gzip')
title_ratings_imdb.head(1)

,tconst,averagerating,numvotes
0,tt10356526,8.3,31


## Rotten Tomatos

### Rotten Tomatos info
# <span style="color:red">  Note: No title, assuming id links to title somewhere</span>

In [138]:
info_rotten_tom = pd.read_csv('zippedData/rt.movie_info.tsv.gz', delimiter='\t', compression='gzip')
info_roten_tom.head(10)

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN
5,8,The year is 1942. As the Allies unite overseas...,PG,Drama|Kids and Family,Jay Russell,Gail Gilchriest,"Mar 3, 2000","Jul 11, 2000",NaN,NaN,95 minutes,Warner Bros. Pictures
6,10,Some cast and crew from NBC's highly acclaimed...,PG-13,Comedy,Jake Kasdan,Mike White,"Jan 11, 2002","Jun 18, 2002",$,"41,032,915",82 minutes,Paramount Pictures
7,13,"Stewart Kane, an Irishman living in the Austra...",R,Drama,Ray Lawrence,Raymond Carver|Beatrix Christian,"Apr 27, 2006","Oct 2, 2007",$,"224,114",123 minutes,Sony Pictures Classics
8,14,"""Love Ranch"" is a bittersweet love story that ...",R,Drama,Taylor Hackford,Mark Jacobson,"Jun 30, 2010","Nov 9, 2010",$,"134,904",117 minutes,NaN
9,15,When a diamond expedition in the Congo is lost...,PG-13,Action and Adventure|Mystery and Suspense|Scie...,Frank Marshall,John Patrick Shanley,"Jun 9, 1995","Jul 27, 1999",NaN,NaN,108 minutes,NaN


### Rotten Tomatos Reviews
# <span style="color:red">  Note: Also no titles, assuming id links to title but don't seem to have that data</span>

In [137]:
reviews_rotten_tom = pd.read_csv('zippedData/rt.reviews.tsv.gz', delimiter='\t', compression='gzip', encoding="cp1252")
reviews_rotten_tom.head(1)

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"


In [135]:
rotten_tom_full = info_rotten_tom.join(reviews_rotten_tom, how='inner')
roten_tom_full.head()

ValueError: columns overlap but no suffix specified: Index(['id', 'rating'], dtype='object')

## The Movie DataBase

### TMDB info

In [77]:
movies_movie_db = pd.read_csv('zippedData/tmdb.movies.csv.gz', compression='gzip')
movies_movie_db.head(1)

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788


### TMDB budgets
# <span style="color:red"> Don't know if the id relates to the TMDB movie information. </span>

In [78]:
movie_budgets = pd.read_csv('zippedData/tn.movie_budgets.csv.gz', compression='gzip')
movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [40]:
reviews_roten_tom.columns

Index(['id', 'review', 'rating', 'fresh', 'critic', 'top_critic', 'publisher',
       'date'],
      dtype='object')